In [1]:
import random
from IPython.display import clear_output

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}
playing = True

In [2]:
class Card:
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return self.rank + " of " + self.suit

In [3]:
class Deck:
    
    def __init__(self):
        
        self.all_cards = []
        for suit in suits:
            for rank in ranks:
                # Create the Card Object
                created_card = Card(suit,rank)
                
                self.all_cards.append(created_card)
                
    def __len__(self):
        len(self.all_cards)
                
    def shuffle(self):        
        random.shuffle(self.all_cards)
        
    def deal_one(self):
        return self.all_cards.pop()
    

In [4]:
class Hand:
    
    def __init__(self,type):
        # type = 'Player' or 'Dealer'
        self.type = type
        self.all_cards = []
        self.value = 0
        self.aces = 0
        
    def __str__(self):
        hand = ''
        for card in self.all_cards[1::]:
            hand = hand + ', ' + str(card)
        if self.value <= 21 and self.type == 'Dealer':
            return f'??{hand}'
        else:
            return f'{str(self.all_cards[0])}{hand}\nValue:\n{self.value}'
    
    def add_one(self,new_card):
        self.all_cards.append(new_card)
        self.value += new_card.value
        if new_card.rank == 'Ace':
            self.aces += 1
            
    def adjust_for_ace(self):
        
        while (self.value > 21) and (self.aces > 0):
            self.value = self.value - 10
            self.aces = self.aces-1
        
    def empty(self):
        self.all_cards = []
        

In [5]:
class Chips:
    
    def __init__(self):
        self.total = 100
        self.bet = 0
        
    def __str__(self):
        return f'{self.total} Chips'
        
    def win_bet(self):
        self.total = self.total + self.bet
    
    def lose_bet(self):
        self.total = self.total - self.bet

In [6]:
def take_bet(chips):
    
    while (chips.total-chips.bet) > 0:
        try:
            bet = int(input(f'How much do you want to bet? Total: {str(chips.total-chips.bet)} chips\n'))
        except:
            Print('Value must be an integer')
        else:
            if bet > (chips.total- chips.bet):
                print(f'Not enough chips: {str(chips.total-chips.bet)} chips total')
                continue
            elif bet < 0:
                print('No negative numbers accepted')
                continue
            else:
                chips.bet += bet
                break

In [7]:
def hit(deck,hand):
    
    hand.add_one(deck.deal_one())
    hand.adjust_for_ace()

In [8]:
def check_empty(deck):
    if len(deck.all_cards) < 1:
        deck = Deck()
        deck.shuffle()
    else:
        pass

In [9]:
def hit_or_stand(deck,hand1,hand2,chips):
    check1 = True
    while hand1.value < 21 and check1:
        take_bet(chips)
        h_or_s = input('Hit or Stand')
        if h_or_s == 'Hit':
            clear_output()
            hit(deck,hand1)
            check_empty(deck)
            print(hand2)
            print(' ')
            print(hand1)
            print(' ')
        elif h_or_s == 'Stand':
            print(f'Player score: {str(hand1.value)}\n')
            check1 = False
        else:
            print('Invalid Answer (Hit or Stand)\n')
        

In [10]:
def player_wins(chips):
    print('Player wins!')
    chips.win_bet()
def dealer_wins(chips):
    print('Dealer wins')
    chips.lose_bet()
def stalemate(chips):
    print('Stalemate')

In [11]:
chips = Chips()
deck1 = Deck()
deck1.shuffle()
    

while playing == True:
    check_empty(deck1)
    # RESETTING THE BET
    chips.bet = 0
    
    # CREATING DEALER HAND AND DEALING TWO CARDS
    hand_dealer = Hand('Dealer')
    hand_dealer.add_one(deck1.deal_one())
    hand_dealer.add_one(deck1.deal_one())
    print("\nDealer's Hand:")
    print(hand_dealer)
    print(' ')
    
    #TAKING BET AND MAKING SURE THE PLAYER
    take_bet(chips)
    print(' ')
    
    while chips.bet:
        
        if chips.bet == 0:
            
            break
        
        # CREATING PLAYER HAND AND DEALING TWO CARDS
        hand_player = Hand('Player')
        hand_player.add_one(deck1.deal_one())
        hand_player.add_one(deck1.deal_one())
        print("Player's Hand:")
        print(hand_player)
        print(' ')
        
        hit_or_stand(deck1,hand_player,hand_dealer,chips)
        
        if hand_player.value > 21:
            print(' ')
            print('\nBUSTED\n')
            dealer_wins(chips)
            print(' ')
            break
        
        if hand_player.value == 21:
            print(' ')
            player_wins(chips)
            print(' ')
            break
        
        hand_dealer.type = 'Show'
        print(hand_dealer)
        while hand_dealer.value < 17 and hand_dealer.value < hand_player.value:
            hit(deck1,hand_dealer)
            check_empty(deck1)
            print('')
            print(hand_dealer)
            print(f'Value: {hand_dealer.value}')
        
        if hand_dealer.value > 21:
            print('\nBUSTED\n')
            player_wins(chips)
            print(' ')
            break
        
        if hand_player.value > hand_dealer.value:
            print(f'\nPlayer: {hand_player.value}\nDealer: {hand_dealer.value}')
            player_wins(chips)
            print(' ')
            break
        
        elif hand_player.value < hand_dealer.value:
            print(f'\nPlayer: {hand_player.value}\nDealer: {hand_dealer.value}')
            dealer_wins(chips)
            print(' ')
            break
            
        elif hand_player.value == hand_dealer.value:
            print(f'\nPlayer: {hand_player.value}\nDealer: {hand_dealer.value}')
            stalemate(chips)
            print(' ')
            break
    
    if chips.total <= 0:
        print('\nNo more Funds!')
        playing = False
        break
    
    answerp = True
    while answerp:
        answer = input('Want to play another round? (y/n)\n')
        
        if answer == 'n':
            answerp = False
            playing = False
        elif answer == 'y':
            answerp = False
        else:
            continue

??, Three of Clubs
 
Ace of Clubs, Five of Spades, Two of Clubs
Value:
18
 


Hit or Stand Stand


Player score: 18

Eight of Clubs, Three of Clubs
Value:
11


NameError: name 'deck' is not defined